<a href="https://colab.research.google.com/github/jhiltonsantos/ADS-Programacao_Internet_I-IFPI/blob/master/buscador.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import requests
import requests_cache
from bs4 import BeautifulSoup
from urllib.request import urlopen
import sqlite3


In [34]:
conn = sqlite3.connect('busca_cache.sqlite')
c = conn.cursor()
  
try:
  c.execute('''CREATE TABLE pages
  (id text, url text, h1 text, p text''') 
  print('nova_tabela')
except:
  requests_cache.install_cache('passa', backend='sqlite')
  print('conexao')


conexao


In [0]:
pages = []
words_page = []


In [36]:
r = requests.get('http://en.wikipedia.org/wiki/Monty_Python')
soup = BeautifulSoup(r.text, 'html.parser')
h1 = soup.find_all('h1')
h1

[<h1 class="firstHeading" id="firstHeading" lang="en">Monty Python</h1>]

In [0]:
# Listas para o banco
url_bd = []
h1 = []
p = []

In [0]:
def download_page(url):
   response = None
   try:
     response = requests.get(url)
     soup = BeautifulSoup(response.text, 'html.parser')
     url_bd.append(url)
     h1.append(soup.find('h1'))
     p.append(soup.find_all('p')[2].get_text())
   except:
     pass
   finally:
     return response
    


In [0]:
def format_link(url, url_or):
  if url != None:
    if url.startswith('http://') or url.startswith('https://'):
      return url
    elif url.startswith('/'):
      return url_or + url
    else:
      return url

In [0]:
def search_links(page, url, depth):
  urls = []
  beatiful = BeautifulSoup(page.text, 'html.parser')

  links = beatiful.find_all('a')

  for link in links:
    if format_link(link.get('href'), url) != None:
      if depth > 0:
        urls.append(format_link(link.get('href'), url))

        url = format_link(link.get('href'), url)
        pages.append({'Url': url, 'Nivel': depth})
  
  return urls

In [0]:
def search_word(keyword, page):
  text = ''
  len_keyword = len(keyword)

  beautiful = BeautifulSoup(page.text, 'html.parser')
  page = beautiful.text
  page = page.replace('\n', '')
  word = page.find(keyword)

  if word < 0:
    text = None
  else:
    if ((page[(word - 10)] == ' ') or (page[(word - 10)] == '\n')):
      text = page[(word):(word+len_keyword)+40]
    else:
      text = page[(word-20) : (word + len_keyword)+20]
  
  return text


In [0]:
def search(keyword, url, depth):
    page = download_page(url)

    if page != None:
      if search_word(keyword, page) != None:
        print('LINK     --> %s' % url)
        print('CONTEUDO --> %s' % search_word(keyword, page))
        words_page.append(url)
      
      if depth > 0:
        urls = search_links(page, url, depth)

        for url in urls:
          try:
           search(keyword, url, depth-1)
          except:
           continue       

In [0]:
def display_search(keyword):
  if len(words_page) > 0:
    print('Busca de %s' %keyword)

    for word in words_page:
      print(word)

In [0]:
add_bd = []

In [45]:
def main():
  keyword = 'artists'
  url = 'https://www.last.fm/home'
  depth = 1

  print("######################################")
  print("Resultado da Busca: ", keyword)

  search(keyword, url, depth)
  
  for i in range(len(url_bd)):
    add_bd.append(url_bd[i])
  
  print(add_bd)
  
  

if __name__ == '__main__':
  main()

  
  




######################################
Resultado da Busca:  artists
LINK     --> https://www.last.fm/home
CONTEUDO --> artists{"@context": "http://schema.org", "@type
LINK     --> https://getsatisfaction.com/lastfm
CONTEUDO --> artists permission and I WANT IT TAKEN DOWN NOW
LINK     --> https://twitter.com/lastfm
CONTEUDO --> ttp://bit.ly/2020topartistsofar … and share #ma
LINK     --> https://www.instagram.com/last_fm
CONTEUDO --> rd with some of the artists. (\ud83d\udcf8: @ri
LINK     --> https://www.youtube.com/user/lastfm
CONTEUDO --> artists right now.    Show moreThis item has be
LINK     --> http://www.last.fm
CONTEUDO --> artists{"@context": "http://schema.org", "@type
['https://www.last.fm/home', 'https://secure.last.fm/login?next=/settings/website', 'https://secure.last.fm/login?next=/settings/website', 'https://secure.last.fm/login', 'https://secure.last.fm/login/dashboard', 'https://secure.last.fm/login/dashboard/music', 'https://secure.last.fm/login/dashboard/music/chart

In [46]:
c.executemany('INSERT INTO pages VALUES(?)', add_bd)

OperationalError: ignored